# Visual Investigation of Mirc

In [ ]:
%matplotlib inline

from scipy.io import loadmat
import scipy
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from visualization import scatter3


points_1 = loadmat('../data/points/mircea_points_1.mat')['points']
points_2 = loadmat('../data/points/mircea_points_2.mat')['points']
points_cad = loadmat('../data/points/603A_CAD.mat')['points']

In [ ]:
def plane_of_points(points, z_min, z_max):
    return points[:, (points[2, :] < z_max) & (points[2, :] > z_min)]

def plot_plane(points, z_min, z_max):
    plane = plane_of_points(points, z_min, z_max)
    plt.scatter(plane[0, :], plane[1, :])

In [ ]:
widgets.interact(lambda mi, ma: plot_plane(points_1, mi, ma), mi=(-100, 100), ma=(-100, 100))

In [ ]:
np.average(points_1, axis=1)

In [ ]:
np.average(points_2, axis=1)

In [ ]:
from registration import build_f
import scipy.optimize

def register_and_plot(A, B, g, rho, tol):
    f = build_f(A, B, g, rho)
    x0 = np.array([0, 0, 0, 0, 0, 0])
    options = {
        'xtol': tol,
        'ftol': 1e-20,  # only care about x
        'maxiter': 4000,
        'return_all': True,
    }
    return scipy.optimize.minimize(f, x0, method='Powell', options=options), f

In [ ]:
g_approx_27_points = lambda mag_b: 0 if mag_b > 29.5 else 1
rho = lambda mag_b: 10
result, f = register_and_plot(points_cad, points_2, g_approx_27_points, rho, tol=1e-8)
print('found x={}, f={}'.format(result.x[:3], f(result.x)))

In [ ]:
xyz = np.array(result.allvecs).T[:3, :]
scatter3({'path': xyz})

In [ ]:
plt.plot([f(v) for v in result.allvecs])

In [ ]:
nx, ny = 20, 20
xl, xh = -4, 4
yl, yh = -4, 4
ff = np.empty((nx, ny), dtype=float)
for xi, xx in enumerate(np.linspace(xl, xh, nx)):
    for yi, yy in enumerate(np.linspace(yl, yh, ny)):
        ff[xi, yi] = f([xx, yy, 1.7, 0*3.14/180, 0, 0])
plt.imshow(ff, interpolation='nearest', extent=(xl, xh, yl, yh), origin='lower')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Min = {}'.format(np.min(ff)))
plt.colorbar()

In [ ]:
testA = np.array([(x, 0, 0) for x in [0, 15.0]], dtype=float).T
testB = np.array([(x, 0, 0) for x in [0, 2, 15.0, 15.0 + 2]]).T
g = lambda b: 1
rho = lambda b: 2.0 + b/15.0
testf = build_f(testA, testB, g, rho)

In [ ]:
testx = np.linspace(-4, 4, 100)
plt.plot(testx, [testf([x, 0, 0, 0, 0, 0]) for x in testx], 'r', label='y=0')
plt.plot(testx, [testf([x, 0.5, 0, 0, 0, 0]) for x in testx], 'g', label='y=0.5')
plt.plot(testx, [testf([x, 1.0, 0, 0, 0, 0]) for x in testx], 'b', label='y=1.0')
plt.plot(testx, [testf([x, 0, 0, 0, 2*3.14/180, 0]) for x in testx], 'b:', label='phi=2deg')